<a href="https://colab.research.google.com/github/kkarimi62/SpaceX-Falcon-9-first-stage-Landing-Prediction/blob/main/BuildDashboardApplicationPlotlyDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2024-06-28 21:32:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2024-06-28 21:32:36 (710 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [2]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.8 MB/s eta 0:00:00


In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


<ipython-input-3-89e3ae8eb04c>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-3-89e3ae8eb04c>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [5]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',options=[{'label':'ALL sites','value':'ALL sites'}]+[{'label':site,'value':site} for site in spacex_df['Launch Site'].unique()],
                      value='ALL sites'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=min_payload,max=max_payload, step=10,
                                                marks={i: '%d'%i for i in range(int(min_payload), int(max_payload) + int(max_payload/10), int(max_payload/10))},
                                                        value=[min_payload, max_payload]

                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown',component_property='value')
    )
def update_graph(selected_site):
    if selected_site == 'ALL sites':
        df_new = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        pie_chart = px.pie(df_new, values='class', names='Launch Site', title='Total Success Launches By Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_counts = filtered_df['class'].value_counts().reset_index()
        success_counts.columns = ['Class', 'Count']
        success_counts['Class'] = success_counts['Class'].map({1: 'Success', 0: 'Failure'})
        pie_chart = px.pie(success_counts, values='Count', names='Class', title=f'Total Success Launches for site {selected_site}')
    return pie_chart

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown',component_property='value'),
     Input(component_id='payload-slider',component_property='value') ],
    )
def update_graph(selected_site, payload_range):
    if selected_site == 'ALL sites':
      filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload_range[0], payload_range[1])]
      scatter_chart = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', #color='Booster Version Category',
                                  title=f'Correlation between Payload and Success for site {selected_site}')

    else:
      filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
      filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(payload_range[0], payload_range[1])]
      scatter_chart = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', #color='Booster Version Category',
                                  title=f'Correlation between Payload and Success for site {selected_site}')
    return scatter_chart

if __name__ == '__main__':
    app.run_server()


<IPython.core.display.Javascript object>